In [1]:
!pip install yfinance psycopg2 pandas


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import yfinance as yf
import psycopg2 
from datetime import datetime

In [4]:
conn = psycopg2.connect(
    dbname="riskanalysispipeline", 
    user="postgres", 
    password="Saniya12", 
    host="localhost", 
    port="5432"
)
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS stocks_data (
                    id SERIAL PRIMARY KEY,
                    ticker VARCHAR(10),
                    date DATE,
                    open FLOAT,
                    high FLOAT,
                    low FLOAT,
                    close FLOAT,
                    volume BIGINT)''')
conn.commit()


In [5]:
def fetch_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    data = stock.history(start=start_date, end=end_date)
    return data

df = fetch_stock_data('AAPL', '2023-01-01', '2023-09-30')
df.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-01-03 00:00:00-05:00,129.066086,129.680305,123.013017,123.904633,112117500,0.0,0.0
2023-01-04 00:00:00-05:00,125.707678,127.461190,123.914545,125.182617,89113600,0.0,0.0
2023-01-05 00:00:00-05:00,125.945420,126.579456,123.597508,123.855080,80962700,0.0,0.0
2023-01-06 00:00:00-05:00,124.835905,129.076017,123.726338,128.412262,87754700,0.0,0.0
2023-01-09 00:00:00-05:00,129.254327,132.166936,128.679730,128.937302,70790800,0.0,0.0


In [6]:
def insert_data_to_db(dataframe, ticker):
    for index, row in dataframe.iterrows():
        cursor.execute('''INSERT INTO stocks_data (ticker, date, open, high, low, close, volume)
                          VALUES (%s, %s, %s, %s, %s, %s, %s)''',
                          (ticker, index.strftime('%Y-%m-%d'), row['Open'], row['High'], row['Low'], row['Close'], row['Volume']))
    conn.commit()

insert_data_to_db(df, 'AAPL')


In [7]:
def calculate_volatility(ticker):
    cursor.execute('SELECT date, close FROM stocks_data WHERE ticker = %s', (ticker,))
    data = cursor.fetchall()

    df = pd.DataFrame(data, columns=['Date', 'Close'])
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    df['Pct_Change'] = df['Close'].pct_change()

    volatility = df['Pct_Change'].std()
    
    return volatility

volatility = calculate_volatility('AAPL')
print(f"Volatility for AAPL: {volatility}")


Volatility for AAPL: 0.013396378616362996


In [1]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import timedelta

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2023, 10, 2),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG('risk_analysis_pipeline', default_args=default_args, schedule_interval='@daily')

def run_pipeline():
    df = fetch_stock_data('AAPL', '2023-01-01', datetime.now().strftime('%Y-%m-%d'))
    insert_data_to_db(df, 'AAPL')
    volatility = calculate_volatility('AAPL')
    print(f"Volatility for AAPL: {volatility}")

run_pipeline_task = PythonOperator(
    task_id='run_pipeline',
    python_callable=run_pipeline,
    dag=dag
)


C:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\airflow\__init__.py:36: RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.
  warnings.warn(


ImportError: DLL load failed while importing _rust: The specified procedure could not be found.